In [8]:
import requests
import pandas as pd
from all_comps import climbing_category, competition_overview_df

# for stage in stages:
url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?'
competition_id = '7923'
category = '5'
stage = 1

# Adding competition, category and route to url
# if category =='5' or category == '6':
url = url + 'comp=' + competition_id + '&cat=' + category + '&route=' + str(stage)
comp_json = requests.get(url).json()

competition_overview_df[competition_overview_df['compId'] == competition_id]

# These ones should definitely be in the overall dataframe, not the climbers one    
    # TODO: Add these directly to the overall competition dataframe
    #     'isoOpen': comp_json.get('route_iso_open'),
    #     'isoClose': comp_json.get('route_iso_close'),
    #     'stageStart': comp_json.get('route_start'),
    #     'numProblems': comp_json.get('route_num_problems'),
    #     'climbingTime': comp_json.get('route_climbing_time'),


# comp_json


,compId,dateEnd,dateStart,hostNation,longCompName,menBouldering,menLead,menSpeed,openComp,otherCategory,shortCompName,womenBouldering,womenLead,womenSpeed
3,7923,2019-04-06,2019-04-05,SUI,IFSC Climbing Worldcup (B) - Meiringen (SUI) 2...,1.0,NaN,NaN,0,1.0,MeiringenWC,1.0,NaN,NaN


In [2]:
def get_stage_results(url_json):
    new_stage_rows = []    
    for climber in comp_json['participants']:
        new_comp_stage = dict()
        
        # This same information needs to be in the row for every climber
        new_comp_stage.update({
            'compId': comp_json.get('WetId'),
            'category': climbing_category(comp_json.get('GrpId')),
            'stage': competition_stage(comp_json['route_order'])
        })

        # Get climber's results for this stage
        new_comp_stage.update({
            'climberID': climber.get('PerId'),
            'startOrder': climber.get('start_order'),
            'rankAfterStage' : climber.get('result_rank'),
            'boulder1': climber.get('boulder1'),
            'boulder2': climber.get('boulder2'),
            'boulder3': climber.get('boulder3'),
            'boulder4': climber.get('boulder4'),
            'boulder5': climber.get('boulder5'),
        })
        
        # Append new row dictionary, (representing a stage of a comp), in the new_stage_rows list
        new_stage_rows.append(new_comp_stage)

    # Return new rows to insert in dataframe
    return new_stage_rows

In [3]:
def competition_stage(stage):
    stages = {
        '-1': 'General result',
        '0': '1. Qualification',
        '1': '2. Qualification',
        '2': 'Semi-Final',
        '3': 'Final'
    }
    return stages.get(stage, 'Other')